# Create ground truth estiamtions of PSI values

Code adapted from `simulate-batcheffect.r` 

In [1]:
library(readr)
library(data.table)
library(dplyr)
library(ggplot2)
library(VGAM)
library(drc)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: stats4

Loading required package: splines

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select



'drc' has been loaded.


Please cite R and 'drc' if used for a publication,

for references type 'citation()' and 'citation('drc')'.



Attaching package: ‘drc’


The following objects are masked from ‘package:VGAM’:

    gompertz, logistic


The following objects are masked from ‘package:stats’:

    gaussian, getInitial



Attaching package: ‘tidyr’


The following object is masked from ‘package:VGAM’:

    fill




## TRex Functions

In [2]:
trex_counts<-function(counts,trex.map,...){
  
    
 trex_sum <- as.data.frame(counts) %>%
                tibble::rownames_to_column("ensembl_transcript_id") %>%
                inner_join(.,trex.map,by = "ensembl_transcript_id") %>%
                dplyr::select(-ensembl_transcript_id) %>%
                group_by(event_id) %>%
                summarize_all(sum)
    
  return(trex_sum)
}

trex_means<-function(lengths,trex.map,...){
  
  trex_mean<-aggregate(lengths[trex.map$ensembl_transcript_id,],
                       list(event_name = trex.map$event_name),
                       function(x){mean(x,na.rm = F)}) %>% 
    tibble::column_to_rownames("event_name")
  return(trex_mean)
}

correct_sim_counts<-function(trex){

    mats<-list()
    for(tc in c("A","C")){
        cols <- grepl(paste0(tc,"_"),colnames(trex$counts))

        # Retrieve counts and length matrices
        countsMat <- trex$counts[,cols]
        lengthMat <- trex$length[,cols] 
        tpmMat <- trex$abundance[,cols]

        # Re-esimate abundances for the trex count type
        tpmSum <- colSums(tpmMat)
        abundanceMat <- t(t(tpmMat)/tpmSum*1e6)

        # Estimate counts from abundances per trex count type
        eCountsMat <- abundanceMat * rowMeans(lengthMat) # Scale to average length over all samples
        scaleFactor <- colSums(countsMat,na.rm=T)/colSums(eCountsMat,na.rm=T)
        eCountsMat <- t(t(eCountsMat) * scaleFactor)

        mats[[tc]] <- list("counts"=eCountsMat,"abundance"=abundanceMat,"length"=lengthMat)
    }
    trex$counts<-cbind(mats[["A"]][["counts"]],mats[["C"]][["counts"]])
    trex$abundance<-cbind(mats[["A"]][["abundance"]],mats[["C"]][["abundance"]])
    trex$length<-cbind(mats[["A"]][["length"]],mats[["C"]][["length"]])
    trex$countsFromAbundance<-"lengthScaledTPM"

    return(trex)
}

create_trex_object<-function(transcript.txi,alt_isos,const_isos){

    message("Converting to event counts...")
    
    tpm<-transcript.txi$abundance
    countsMat<-transcript.txi$counts
    lengthMat<-transcript.txi$length
    
    tpm_alt<-trex_counts(tpm,alt_isos) 
    colnames(tpm_alt)[-1]<-paste0("A_",colnames(tpm_alt)[-1])

    tpm_const<-trex_counts(tpm,const_isos)
    colnames(tpm_const)[-1]<-paste0("C_",colnames(tpm_const)[-1])

    cts_alt<-trex_counts(countsMat,alt_isos)
    colnames(cts_alt)[-1]<-paste0("A_",colnames(cts_alt)[-1])

    cts_const<-trex_counts(countsMat,const_isos) 
    colnames(cts_const)[-1]<-paste0("C_",colnames(cts_const)[-1]) 
    
    len_alt<-inner_join(alt_isos,lengthMat %>% tibble::rownames_to_column("ensembl_transcript_id"),by = "ensembl_transcript_id") %>%
                group_by(event_id) %>% dplyr::select(-ensembl_transcript_id) %>%
                summarize_all(mean)
    colnames(len_alt)[-1]<-paste0("A_",colnames(len_alt)[-1])

    len_const<-inner_join(const_isos,lengthMat %>% tibble::rownames_to_column("ensembl_transcript_id"),by = "ensembl_transcript_id") %>%
                     group_by(event_id) %>% dplyr::select(-ensembl_transcript_id) %>%
                    summarize_all(mean)
    colnames(len_const)[-1]<-paste0("C_",colnames(len_const)[-1])

    trex<-list()
    trex$abundance<-inner_join(tpm_alt,tpm_const,by="event_id") %>% tibble::column_to_rownames("event_id")
    trex$counts<-inner_join(cts_alt,cts_const,by="event_id") %>% tibble::column_to_rownames("event_id")
    trex$length<-inner_join(len_alt,len_const,by="event_id") %>% tibble::column_to_rownames("event_id")
    
    trex<-correct_sim_counts(trex)
    
    return(trex)
}

## Load data

In [3]:
dir_input <- "/project/6007998/lmoral7/altsplicing-methods/simulations/data/observed_counts"
effect_dir <- "/project/6007998/lmoral7/altsplicing-methods/simulations/data/sampled_counts/gt_effects"
asmapfile <- "/home/lmoral7/lmprojects/references/gencode_GRCh38.p13/SUPPA2_splicemap/modID.GRCh38.p13.gencode.v37.primary_assembly.annotation_ASALL_strict.ioe"
sample_file <- "K562_control_rep1/rsem.isoforms.results"
sample_file_2 <- "K562_SRSF9.KD_rep1/rsem.isoforms.results"

In [4]:
## Splicing GTF map
asmap<-fread(asmapfile,data.table = F,stringsAsFactors = F) %>% 
        as.data.frame() %>% 
        filter(event_type=="SE") %>% distinct() 
rownames(asmap)<-asmap$event_id

In [5]:
## Ground truth counts 

iso_res_rsem<-read.table(file.path(dir_input,sample_file),header=TRUE)
iso_res_kd_rsem<-read.table(file.path(dir_input,sample_file_2),header=TRUE)
obs_TPM <- fread(file.path(dir_input,"ground_truth_TPM.tsv"),data.table=FALSE)
simulation_design <- fread(file.path(dir_input,"metadata.tsv"))

n_transcripts <- nrow(obs_TPM)
pseudocount <- 1e-3
set.seed(7)

In [6]:
nonZero_TPMs <- rowSums( obs_TPM > 0 ) 
lengthMat <-  inner_join(iso_res_rsem[,c("transcript_id","effective_length")]  %>% rename("K562_control"="effective_length") ,
                         iso_res_kd_rsem[,c("transcript_id","effective_length")] %>% rename("K562_SRSF9.KD"="effective_length")) %>%
              tibble::column_to_rownames("transcript_id")
countsMat <- inner_join(iso_res_rsem[,c("transcript_id","expected_count")]  %>% rename("K562_control"="expected_count") ,
                         iso_res_kd_rsem[,c("transcript_id","expected_count")] %>% rename("K562_SRSF9.KD"="expected_count")) %>%
              tibble::column_to_rownames("transcript_id")

lengthMat<-cbind(lengthMat,lengthMat,lengthMat,lengthMat,lengthMat)
colnames(lengthMat)<-paste(colnames(lengthMat),rep(paste0("rep",1:5),each = 2),sep="_")
lengthMat<-lengthMat[,simulation_design$Sample]

countsMat<-cbind(countsMat,countsMat,countsMat,countsMat,countsMat)
colnames(countsMat)<-paste(colnames(countsMat),rep(paste0("rep",1:5),each = 2),sep="_")
countsMat<-countsMat[,simulation_design$Sample]

Joining, by = "transcript_id"
Joining, by = "transcript_id"


In [7]:
## Ground truth effect sizes 
pars <- list.files(effect_dir)
gt_effects <- lapply(pars, function(p,...){
    ref<-read.table(file = file.path(effect_dir,p),sep = "\t")
})
names(gt_effects)<-sub("_ground_truth_effects.tsv","",pars)

In [8]:
## Assign events to isoforms 
alt_isos<-asmap %>%
        group_by(event_id)%>%
        mutate(ensembl_transcript_id=strsplit(alternative_transcripts,",")) %>%
        tidyr::unnest(ensembl_transcript_id) %>%
        dplyr::select(event_id,ensembl_transcript_id) %>% ungroup()

const_isos<-asmap %>%
            group_by(event_id)%>%
            mutate(ensembl_transcript_id=strsplit(constitutive_transcripts,",")) %>%
            tidyr::unnest(ensembl_transcript_id) %>%
            dplyr::select(event_id,ensembl_transcript_id) %>% ungroup()

## Ground truth effects 

In [9]:
# Generate simulated log TPM
pseudocount<-1e-6
design_mx <- model.matrix( ~ cell_line + batch, simulation_design )

In [10]:
gt.effects_to_gt.PSI<-function(gt_effects_mat,...){
    
    message("Generating logTPM...")
    gt_effects_mat <- as.matrix(gt_effects_mat)
    logTPM <- gt_effects_mat %*% t(design_mx)
    tpm<-exp(logTPM)
    tpm[nonZero_TPMs==0,] <- 0
    tpm <- scale(tpm, center = F, scale=colSums(tpm)/1000000)
    colnames(tpm)<-simulation_design$Sample
    
    # Get event counts 
    txi<-list("abundance"=tpm,
              "counts"=countsMat,
             "length"=lengthMat)
    trex<-create_trex_object(txi,alt_isos,const_isos) 
    
    message("Regressing out effect sizes on PSI")
    tpm_alt<-trex$abundance[,grepl("A_",colnames(trex$abundance))]
    tpm_const<-trex$abundance[,grepl("C_",colnames(trex$abundance))]
    
    psi<-tpm_alt/(tpm_alt+tpm_const)
    psi<-t(log(psi/(1-psi))) #change to logit psi
    psi[is.na(psi)|is.infinite(psi)]<-log(pseudocount/(1-pseudocount))
    
    psi.fit<-lm(psi ~ cell_line + batch,data = simulation_design)
    
    return(t(psi.fit$coefficients))
    
}

In [11]:
psi_effects<-lapply(gt_effects,gt.effects_to_gt.PSI)

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...

Converting to event counts...

Regressing out effect sizes on PSI

Generating logTPM...


In [12]:
saveRDS(psi_effects,file = "../data/ground_truth_effect_sizes_on_lenCorrected_logitPSI.RData")